In [16]:
import pandas as pd

df = pd.read_csv(r"..\data\raw\nutritional-facts.csv")
df.fillna(0, inplace=True)
df.head()

,Food Name,Category Name,Calcium,Calories,Carbs,Cholesterol,Copper,Fats,Fiber,Folate,...,Vitamin D,Vitamin E,Vitamin K,Omega-3 - ALA,Omega-6 - Eicosadienoic acid,Omega-6 - Gamma-linoleic acid,Omega-3 - Eicosatrienoic acid,Omega-6 - Dihomo-gamma-linoleic acid,Omega-6 - Linoleic acid,Omega-6 - Arachidonic acid
0,Apple,Fruits,0.006,52.0,14.0,0.0,0.00003,0.17,2.4,0.000003,...,0.0,0.00018,2.200000e-06,0.00,0.0,0.00,0.0,0.0,0.0,0.0
1,Apricot,Fruits,0.013,48.0,11.0,0.0,0.00008,0.39,2.0,0.000009,...,0.0,0.00089,3.300000e-06,0.00,0.0,0.00,0.0,0.0,0.0,0.0
2,Dried fruit,Fruits,0.055,241.0,63.0,0.0,0.00034,0.51,7.3,0.000010,...,0.0,0.00430,3.100000e-06,0.00,0.0,0.00,0.0,0.0,0.0,0.0
3,Avocado,Fruits,0.012,160.0,8.5,0.0,0.00019,15.00,6.7,0.000081,...,0.0,0.00210,2.100000e-05,0.11,0.0,0.02,0.0,0.0,0.0,0.0
4,Banana,Fruits,0.005,89.0,23.0,0.0,0.00008,0.33,2.6,0.000020,...,0.0,0.00010,5.000000e-07,0.00,0.0,0.00,0.0,0.0,0.0,0.0


In [17]:
import json, datetime

with open('../data/raw/food-seasonality.json', 'r') as f:
    seasonality_data = json.load(f)

def find_seasonal_food(seasonality_data: any, month: str = ""):
    fruits = []
    vegetables = []

    if month == "":
        current_month = datetime.datetime.now().strftime("%m")
    else:
        current_month = month

    seasonal_foods = seasonality_data["Italy"].get(current_month, [])

    for food in seasonal_foods:
        food_category = df[df['Food Name'] == food]['Category Name'].values
        if len(food_category) > 0:
            if food_category[0] == "Fruits":
                fruits.append(food)
            else:
                vegetables.append(food)

    return fruits, vegetables

fruits, vegetables = find_seasonal_food(seasonality_data)
print("Seasonal fruits: ", fruits)
print("Seasonal vegetables: ", vegetables)

Seasonal fruits:  ['Orange', 'Clementine', 'Kiwifruit', 'Mandarin orange', 'Apple', 'Pear', 'Grapefruit']
Seasonal vegetables:  ['Artichoke', 'Carrot', 'Broccoli', 'Cauliflower', 'Cabbage', 'Chicory', 'Fennel', 'Potato', 'Radicchio', 'Turnip', 'Spinach', 'Pumpkin']


In [18]:
def find_food_nutritional_info(food_name: str, only_numbers: bool = True):
    food_info = df[df['Food Name'] == food_name]
    if len(food_info) > 0:
        if only_numbers:
            return food_info.drop(columns=['Food Name', 'Category Name'])
        return food_info
    else:
        return "No information found for this food"
    
food_name = "Apple"
food_info = find_food_nutritional_info(food_name)
food_info

,Calcium,Calories,Carbs,Cholesterol,Copper,Fats,Fiber,Folate,Iron,Magnesium,...,Vitamin D,Vitamin E,Vitamin K,Omega-3 - ALA,Omega-6 - Eicosadienoic acid,Omega-6 - Gamma-linoleic acid,Omega-3 - Eicosatrienoic acid,Omega-6 - Dihomo-gamma-linoleic acid,Omega-6 - Linoleic acid,Omega-6 - Arachidonic acid
0,0.006,52.0,14.0,0.0,0.00003,0.17,2.4,0.000003,0.00012,0.005,...,0.0,0.00018,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
def compute_energy_density_per_100g(food_name):
    # Low energy density (< 1.5 kcal/g)
    # Medium energy density (1.5 - 2.5 kcal/g)
    # High energy density (> 2.5 kcal/g)
    food_info = find_food_nutritional_info(food_name)
    if isinstance(food_info, str):
        return food_info
    energy_density = food_info['Calories'].values[0] / 100

    if energy_density < 1.5:
        return ("Low", energy_density)
    elif 1.5 <= energy_density <= 2.5:
        return ("Medium", energy_density)
    else:
        return ("High", energy_density)

energy_density = compute_energy_density_per_100g(food_name="Ricotta")
energy_density

('Medium', np.float64(1.74))

In [20]:
import numpy as np
from numpy.linalg import norm

def find_similar_food(df: pd.DataFrame, food_name: str, n: int = 1, same_category: bool = True, low_density_food: bool = True):
    similar_foods = []

    if same_category:
        food_info = df[df['Food Name'] == food_name]
        if food_info.empty:
            return f"No information found for food: {food_name}"
        
        category = food_info['Category Name'].values[0]
        food_A = find_food_nutritional_info(food_name, only_numbers=True).to_numpy().flatten()
        
        for food in df[df['Category Name'] == category]['Food Name']:
            if food == food_name:
                continue
            food_B = find_food_nutritional_info(food, only_numbers=True).to_numpy().flatten()
            norm_A = norm(food_A)
            norm_B = norm(food_B)

            if norm_A == 0 or norm_B == 0:
                similarity = 0
            else:
                similarity = np.dot(food_A, food_B) / (norm_A * norm_B)
            similar_foods.append((food, similarity))

    if low_density_food == False or compute_energy_density_per_100g(food_name)[0] == "Low":
        similar_foods = sorted(similar_foods, key=lambda x: x[1], reverse=True)
    else:
        similar_foods = sorted(similar_foods, key=lambda x: x[1], reverse=True)
        similar_foods = sorted(similar_foods, key=lambda x: (compute_energy_density_per_100g(x[0])[1], x[1]))
        similar_foods = [(food, similarity, compute_energy_density_per_100g(food)[0]) for food, similarity in similar_foods]
        
    return similar_foods

similar_foods = find_similar_food(df, food_name="Apple")
print(similar_foods[0:3])

[('Blueberry', np.float64(0.9988352415031658)), ('Pineapple', np.float64(0.9970988211193896)), ('Honeydew', np.float64(0.9891949873765672))]


In [21]:
# how to print low density foods
low_density_foods = []
for food in df['Food Name']:
    density = compute_energy_density_per_100g(food)
    if density[0] == "Low":
        low_density_foods.append((food, density[1]))

In [22]:
def filter_by_diet_and_intolerances(df, diet='omnivore', gluten_free=False, dairy_free=False):
    # Create a copy of the dataframe
    filtered_df = df.copy()
    
    # Define categories to exclude based on diet
    exclude_categories = []
    if diet == 'vegan':
        exclude_categories.extend(['Dairy', 'Lactose-Free Dairy', 'Cured Meat', 'Red Meat', 'White Meat', 'Seafood', 'Eggs'])
    elif diet == 'vegetarian':
        exclude_categories.extend(['Cured Meat', 'Red Meat', 'White Meat', 'Seafood'])
    
    # Add dairy categories if dairy-free
    if dairy_free:
        exclude_categories.extend(['Dairy', 'Lactose-Free Dairy'])
    
    # Filter out excluded categories
    filtered_df = filtered_df[~filtered_df['Category Name'].isin(exclude_categories)]
    
    # Filter out gluten-containing foods if gluten-free
    if gluten_free:
        gluten_foods = ['Wheat Bread', 'Pasta']  # Add more gluten-containing foods as needed
        filtered_df = filtered_df[~filtered_df['Food Name'].isin(gluten_foods)]
    
    return filtered_df.reset_index(drop=True)

# ask the user and try the function
diet = input("Please enter your diet (omnivore, vegetarian, vegan): ")
gluten_free = input("Are you gluten-free? (yes/no): ").lower() == 'yes'
dairy_free = input("Are you dairy-free? (yes/no): ").lower() == 'yes'

filtered_df = filter_by_diet_and_intolerances(df, diet=diet, gluten_free=gluten_free, dairy_free=dairy_free)
filtered_df.head()

,Food Name,Category Name,Calcium,Calories,Carbs,Cholesterol,Copper,Fats,Fiber,Folate,...,Vitamin D,Vitamin E,Vitamin K,Omega-3 - ALA,Omega-6 - Eicosadienoic acid,Omega-6 - Gamma-linoleic acid,Omega-3 - Eicosatrienoic acid,Omega-6 - Dihomo-gamma-linoleic acid,Omega-6 - Linoleic acid,Omega-6 - Arachidonic acid
0,Apple,Fruits,0.006,52.0,14.0,0.0,0.00003,0.17,2.4,0.000003,...,0.0,0.00018,2.200000e-06,0.00,0.0,0.00,0.0,0.0,0.0,0.0
1,Apricot,Fruits,0.013,48.0,11.0,0.0,0.00008,0.39,2.0,0.000009,...,0.0,0.00089,3.300000e-06,0.00,0.0,0.00,0.0,0.0,0.0,0.0
2,Dried fruit,Fruits,0.055,241.0,63.0,0.0,0.00034,0.51,7.3,0.000010,...,0.0,0.00430,3.100000e-06,0.00,0.0,0.00,0.0,0.0,0.0,0.0
3,Avocado,Fruits,0.012,160.0,8.5,0.0,0.00019,15.00,6.7,0.000081,...,0.0,0.00210,2.100000e-05,0.11,0.0,0.02,0.0,0.0,0.0,0.0
4,Banana,Fruits,0.005,89.0,23.0,0.0,0.00008,0.33,2.6,0.000020,...,0.0,0.00010,5.000000e-07,0.00,0.0,0.00,0.0,0.0,0.0,0.0


In [23]:
def get_user_preferences(df: pd.DataFrame, filtered_df: pd.DataFrame):
    # Define categories to collect preferences for
    categories = [
        "Grains", "Gluten-Free Grains", "Legumes", "Dairy", "Lactose-Free Dairy",
        "Cured Meat", "Red Meat", "White Meat", "Seafood",
        "Oils", "Sauces", "Eggs"
    ]

    italian_preferences = {
        "Grains": ["Pasta", "Wheat Bread"],
        "Gluent-Free Grains": ["Rice"],
        "Legumes": ["Chickpeas", "Lentils"],
        "Dairy": ["Mozzarella",  "Ricotta"],
        "Lactose-Free Dairy": ["Parmigiano-Reggiano"],
        "Cured Meat": ["Mortadella", "Salami"],
        "Red Meat": ["Steak"],
        "White Meat": ["Chicken meat"],
        "Seafood": ["Tuna", "Salmon"],
        "Eggs": ["Eggs"],
        "Oils": ["Olive oil"],
        "Sauces": ["Tomato Sauce", "Salsa"]
    }
    
    preferences = {}
    
    print("Please select your preferred foods for each category.")
    print("Enter numbers separated by spaces, or press Enter to skip.")
    
    for category in categories:
        # Get foods for this category from filtered_df (respecting dietary restrictions)
        foods = filtered_df[filtered_df["Category Name"] == category]["Food Name"].tolist()
        
        if not foods:  # Skip empty categories (due to dietary restrictions)
            continue
            
        # Show options
        print(f"\n{category}:")
        for i, food in enumerate(foods, 1):
            print(f"{i}. {food}")
        
        # Get user input
        while True:
            try:
                choices = input(f"Select {category} (1-{len(foods)}): ").strip()
                if not choices:  # If no selection, include all foods from this category
                    preferences[category] = italian_preferences[category]
                    break
                    
                # Convert input to list of integers
                selected = [int(x) for x in choices.split()]
                
                # Validate choices
                if all(1 <= x <= len(foods) for x in selected):
                    preferences[category] = [foods[i-1] for i in selected]
                    break
                else:
                    print("Invalid selection. Please try again.")
            except ValueError:
                print("Please enter valid numbers separated by spaces.")

    # Handle cold start: if no preferences were selected, use filtered_df
    if not preferences:
        return filtered_df
    
    # Combine all preferences into a single dataframe
    combined_preferences = pd.concat([filtered_df[filtered_df["Food Name"].isin(preferences[category])] 
                                    for category in preferences])
    
    # If combined_preferences is empty, fallback to filtered_df
    if combined_preferences.empty:
        return filtered_df
        
    return combined_preferences.reset_index(drop=True)

user_preferences = get_user_preferences(df, filtered_df)
user_preferences

Please select your preferred foods for each category.
Enter numbers separated by spaces, or press Enter to skip.

Grains:
1. Wheat Bread
2. Couscous
3. Pasta

Gluten-Free Grains:
1. Buckwheat
2. Millet
3. Oat
4. Rice

Legumes:
1. Chickpeas
2. Lentil
3. Soybean
4. Tempeh
5. Hummus
6. Tofu
7. Bean

Dairy:
1. Cheese
2. Cottage cheese
3. Cream cheese
4. Ricotta
5. Roquefort
6. Goat cheese
7. Mozzarella

Lactose-Free Dairy:
1. Blue cheese
2. Brie
3. Camembert
4. Edam
5. Feta
6. Fontina
7. Gouda cheese
8. Gruyere cheese
9. Provolone
10. Romano cheese
11. Swiss cheese
12. Parmigiano-Reggiano
13. Margarine

Cured Meat:
1. Turkey ham
2. Luncheon meat
3. Mortadella
4. Salami
5. Italian sausage
6. Turkey bacon
7. Turkey sausage
8. Ham
9. Pork bacon

Red Meat:
1. Roast beef
2. Meatball
3. Pork belly
4. Pork
5. Ground beef
6. Goat
7. Steak
8. Beef

White Meat:
1. Chicken meat
2. Turkey meat
3. Ground turkey
4. Ground chicken
5. Lamb
6. Rabbit Meat

Seafood:
1. Anchovy
2. Carp
3. Cod
4. Fish sticks


,Food Name,Category Name,Calcium,Calories,Carbs,Cholesterol,Copper,Fats,Fiber,Folate,...,Vitamin D,Vitamin E,Vitamin K,Omega-3 - ALA,Omega-6 - Eicosadienoic acid,Omega-6 - Gamma-linoleic acid,Omega-3 - Eicosatrienoic acid,Omega-6 - Dihomo-gamma-linoleic acid,Omega-6 - Linoleic acid,Omega-6 - Arachidonic acid
0,Wheat Bread,Grains,0.165,313.0,56.0,0.000,0.00020,4.30,4.7,0.000086,...,0.000000e+00,0.00024,5.700000e-06,0.16,0.00,0.00,0.00,0.00,0.00,0.0
1,Pasta,Grains,0.006,131.0,25.0,0.033,0.00009,1.10,0.0,0.000064,...,0.000000e+00,0.00000,0.000000e+00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
2,Rice,Gluten-Free Grains,0.010,130.0,28.0,0.000,0.00007,0.28,0.4,0.000058,...,0.000000e+00,0.00004,0.000000e+00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
3,Chickpeas,Legumes,0.049,164.0,27.0,0.000,0.00035,2.60,7.6,0.000172,...,0.000000e+00,0.00035,4.000000e-06,0.00,0.00,0.00,0.00,0.00,0.00,0.0
4,Lentil,Legumes,0.019,116.0,20.0,0.000,0.00025,0.38,7.9,0.000181,...,0.000000e+00,0.00011,1.700000e-06,0.00,0.00,0.00,0.00,0.00,0.00,0.0
5,Bean,Legumes,0.050,105.0,20.0,0.000,0.00021,0.40,5.5,0.000024,...,0.000000e+00,0.00015,8.000000e-07,0.00,0.00,0.00,0.00,0.00,0.00,0.0
6,Cheese,Dairy,0.710,404.0,3.1,0.099,0.00003,33.00,0.0,0.000027,...,6.000000e-07,0.00071,2.400000e-06,0.00,0.01,0.00,0.00,0.00,0.00,0.0
7,Mozzarella,Dairy,0.731,280.0,3.1,0.054,0.00003,17.00,0.0,0.000009,...,3.000000e-07,0.00015,1.800000e-06,0.00,0.00,0.00,0.00,0.00,0.00,0.0
8,Romano cheese,Lactose-Free Dairy,1.064,387.0,3.6,0.104,0.00003,27.00,0.0,0.000007,...,5.000000e-07,0.00023,2.200000e-06,0.00,0.00,0.00,0.00,0.00,0.00,0.0
9,Parmigiano-Reggiano,Lactose-Free Dairy,1.109,265.0,1.4,0.088,0.00024,20.00,0.0,0.000010,...,4.000000e-07,0.00017,1.700000e-06,0.00,0.00,0.00,0.00,0.00,0.00,0.0


In [24]:
# Get current month
current_month = datetime.datetime.now().strftime("%m")

# Find seasonal fruits and vegetables
seasonal_fruits, seasonal_vegetables = find_seasonal_food(seasonality_data, current_month)

# Ask the user for their preferred seasonal fruits and vegetables
print("Please select your preferred seasonal fruits and vegetables for this month.")
print("Enter numbers separated by spaces, or press Enter to skip.")

# Get user preferences for seasonal fruits
print("\nSeasonal Fruits:")
for i, fruit in enumerate(seasonal_fruits, 1):
    print(f"{i}. {fruit}")

selected_fruits = input(f"Select fruits (1-{len(seasonal_fruits)}): ").strip()
if selected_fruits:
    selected_fruits = [seasonal_fruits[int(i)-1] for i in selected_fruits.split()]
else:
    selected_fruits = seasonal_fruits

# Get user preferences for seasonal vegetables
print("\nSeasonal Vegetables:")
for i, vegetable in enumerate(seasonal_vegetables, 1):
    print(f"{i}. {vegetable}")

selected_vegetables = input(f"Select vegetables (1-{len(seasonal_vegetables)}): ").strip()
if selected_vegetables:
    selected_vegetables = [seasonal_vegetables[int(i)-1] for i in selected_vegetables.split()]
else:
    selected_vegetables = seasonal_vegetables

# Filter the user preferences dataframe to include only selected seasonal fruits and vegetables
selected_fruits_df = df[df['Food Name'].isin(selected_fruits)]
selected_vegetables_df = df[df['Food Name'].isin(selected_vegetables)]

# Combine selected seasonal fruits and vegetables with user preferences
user_preferences = pd.concat([user_preferences, selected_fruits_df, selected_vegetables_df]).drop_duplicates().reset_index(drop=True)

user_preferences.head()

Please select your preferred seasonal fruits and vegetables for this month.
Enter numbers separated by spaces, or press Enter to skip.

Seasonal Fruits:
1. Orange
2. Clementine
3. Kiwifruit
4. Mandarin orange
5. Apple
6. Pear
7. Grapefruit

Seasonal Vegetables:
1. Artichoke
2. Carrot
3. Broccoli
4. Cauliflower
5. Cabbage
6. Chicory
7. Fennel
8. Potato
9. Radicchio
10. Turnip
11. Spinach
12. Pumpkin


,Food Name,Category Name,Calcium,Calories,Carbs,Cholesterol,Copper,Fats,Fiber,Folate,...,Vitamin D,Vitamin E,Vitamin K,Omega-3 - ALA,Omega-6 - Eicosadienoic acid,Omega-6 - Gamma-linoleic acid,Omega-3 - Eicosatrienoic acid,Omega-6 - Dihomo-gamma-linoleic acid,Omega-6 - Linoleic acid,Omega-6 - Arachidonic acid
0,Wheat Bread,Grains,0.165,313.0,56.0,0.000,0.00020,4.30,4.7,0.000086,...,0.0,0.00024,0.000006,0.16,0.0,0.0,0.0,0.0,0.0,0.0
1,Pasta,Grains,0.006,131.0,25.0,0.033,0.00009,1.10,0.0,0.000064,...,0.0,0.00000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,Rice,Gluten-Free Grains,0.010,130.0,28.0,0.000,0.00007,0.28,0.4,0.000058,...,0.0,0.00004,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,Chickpeas,Legumes,0.049,164.0,27.0,0.000,0.00035,2.60,7.6,0.000172,...,0.0,0.00035,0.000004,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,Lentil,Legumes,0.019,116.0,20.0,0.000,0.00025,0.38,7.9,0.000181,...,0.0,0.00011,0.000002,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
import random

def generate_meal(user_preferences: pd.DataFrame = user_preferences, filtered_df: pd.DataFrame = filtered_df):
    meal = []
    meal.extend([
        random.choice(
            user_preferences[user_preferences["Category Name"] == "Grains"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Gluten-Free Grains"]["Food Name"].to_list()
        ),
        random.choice(
            user_preferences[user_preferences["Category Name"] == "Legumes"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Dairy"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Lactose-Free Dairy"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Cured Meat"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Red Meat"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "White Meat"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Seafood"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Eggs"]["Food Name"].to_list()
        ),
        random.choice(user_preferences[user_preferences["Category Name"] == "Oils"]["Food Name"].to_list()),
        random.choice(user_preferences[user_preferences["Category Name"] == "Sauces"]["Food Name"].to_list()),
        random.choice(user_preferences[user_preferences["Category Name"] == "Vegetables"]["Food Name"].to_list()),
        random.choice(user_preferences[user_preferences["Category Name"] == "Fruits"]["Food Name"].to_list())
    ])
    
    # Find similar foods for each item in the meal
    similar_meal = []
    for item in meal:
        # Get the category of the current item
        item_category = user_preferences[user_preferences["Food Name"] == item]["Category Name"].values[0]
        
        # Only find similar foods if not a fruit or vegetable
        if item_category not in ["Fruits", "Vegetables", "Greens"]:
            similar_foods = find_similar_food(filtered_df, food_name=item, n=1, same_category=True, low_density_food=True)
            if similar_foods:
                similar_meal.append(similar_foods[0][0])
            else:
                similar_meal.append(item)
        else:
            similar_meal.append(item)
    
    return meal, similar_meal

generate_meal(user_preferences, filtered_df)

(['Pasta', 'Steak', 'Olive oil', 'Mustard', 'Carrot', 'Clementine'],
 ['Couscous',
  'Roast beef',
  'Butter',
  'Barbecue sauce',
  'Carrot',
  'Clementine'])

In [41]:
import random

# Load food servings data
with open('../data/raw/food-servings.json', 'r') as f:
    food_servings = json.load(f)

# Define servings count
servings_count = food_servings.copy()

# Generate meals based on frequency per week
generated_meals = []
for category, info in servings_count.items():
    if category in ["Grains", "Gluten-Free Grains", "Fruits", "Vegetables", "Beverages", "Nuts", "Greens", "Oils", "Sauces"]:
        continue
    for _ in range(info["frequency_per_week"]):
        meal = []
        meal.extend([
            random.choice(
                user_preferences[user_preferences["Category Name"] == "Grains"]["Food Name"].to_list() +
                user_preferences[user_preferences["Category Name"] == "Gluten-Free Grains"]["Food Name"].to_list()
            ),
            random.choice(user_preferences[user_preferences["Category Name"] == category]["Food Name"].to_list()),
            random.choice(user_preferences[user_preferences["Category Name"] == "Oils"]["Food Name"].to_list()),
            random.choice(user_preferences[user_preferences["Category Name"] == "Sauces"]["Food Name"].to_list()),
            random.choice(user_preferences[user_preferences["Category Name"] == "Vegetables"]["Food Name"].to_list()),
            random.choice(user_preferences[user_preferences["Category Name"] == "Fruits"]["Food Name"].to_list())
        ])
        
        similar_meal = []
        for item in meal:
            item_category = user_preferences[user_preferences["Food Name"] == item]["Category Name"].values[0]
            if item_category not in ["Fruits", "Vegetables", "Greens"]:
                similar_foods = find_similar_food(filtered_df, food_name=item, n=1, same_category=True, low_density_food=True)
                if similar_foods:
                    similar_meal.append(similar_foods[0][0])
                else:
                    similar_meal.append(item)
            else:
                similar_meal.append(item)
        
        generated_meals.append((meal, similar_meal))


    # print(f"Generated {info['frequency_per_week']} meals for {category}.")

# Randomly select 14 meals from the generated meals
weekly_meals = random.sample(generated_meals, 14)

# Split the meals into 7 lunches and 7 dinners
lunches = weekly_meals[:7]
dinners = weekly_meals[7:]

# Print the generated meals for the week
print("Lunches:")
for day, (meal, similar_meal) in enumerate(lunches, 1):
    print(f"Day {day}:")
    print(f"  Meal: {meal}")
    print(f"  Similar Meal: {similar_meal}")
    print()

print("Dinners:")
for day, (meal, similar_meal) in enumerate(dinners, 1):
    print(f"Day {day}:")
    print(f"  Meal: {meal}")
    print(f"  Similar Meal: {similar_meal}")
    print()

Lunches:
Day 1:
  Meal: ['Pasta', 'Bean', 'Olive oil', 'Tomato sauce', 'Carrot', 'Orange']
  Similar Meal: ['Couscous', 'Tofu', 'Butter', 'Tomato paste', 'Carrot', 'Orange']

Day 2:
  Meal: ['Pasta', 'Parmigiano-Reggiano', 'Olive oil', 'Mustard', 'Fennel', 'Orange']
  Similar Meal: ['Couscous', 'Feta', 'Butter', 'Barbecue sauce', 'Fennel', 'Orange']

Day 3:
  Meal: ['Pasta', 'Ham', 'Olive oil', 'Barbecue sauce', 'Turnip', 'Clementine']
  Similar Meal: ['Couscous', 'Luncheon meat', 'Butter', 'Tomato sauce', 'Turnip', 'Clementine']

Day 4:
  Meal: ['Rice', 'Chicken meat', 'Olive oil', 'Tomato sauce', 'Turnip', 'Orange']
  Similar Meal: ['Millet', 'Rabbit Meat', 'Butter', 'Tomato paste', 'Turnip', 'Orange']

Day 5:
  Meal: ['Pasta', 'Egg', 'Olive oil', 'Soy sauce', 'Turnip', 'Kiwifruit']
  Similar Meal: ['Couscous', 'Egg white', 'Butter', 'Barbecue sauce', 'Turnip', 'Kiwifruit']

Day 6:
  Meal: ['Pasta', 'Chicken meat', 'Olive oil', 'Barbecue sauce', 'Turnip', 'Kiwifruit']
  Similar Meal:

In [27]:
# Generate a full week of meals
weekly_meals = []
for _ in range(7):
    meal, similar_meal = generate_meal(user_preferences, filtered_df)
    weekly_meals.append((meal, similar_meal))

# Print the generated meals for the week
for day, (meal, similar_meal) in enumerate(weekly_meals, 1):
    print(f"Day {day}:")
    print(f"  Meal: {meal}")
    print(f"  Similar Meal: {similar_meal}")
    print()

# Load food servings data
with open('../data/raw/food-servings.json', 'r') as f:
    food_servings = json.load(f)

# Define servings count
servings_count = food_servings.copy()

# Adjust the weekly meals based on servings count and frequency
adjusted_weekly_meals = []
for day, (meal, similar_meal) in enumerate(weekly_meals, 1):
    adjusted_meal = []
    for food in meal:
        category = user_preferences[user_preferences['Food Name'] == food]['Category Name'].values[0]
        if servings_count[category]["frequency_per_week"] > 0:
            adjusted_meal.append(food)
            servings_count[category]["frequency_per_week"] -= 1
        else:
            # Generate a new meal if frequency exceeded
            new_meal, _ = generate_meal(user_preferences, filtered_df)
            adjusted_meal.extend(new_meal)
            break
        weekly_meals[day-1] = (adjusted_meal, similar_meal)


Day 1:
  Meal: ['Wheat Bread', 'Beef', 'Olive oil', 'Mustard', 'Turnip', 'Orange']
  Similar Meal: ['Couscous', 'Roast beef', 'Butter', 'Barbecue sauce', 'Turnip', 'Orange']

Day 2:
  Meal: ['Wheat Bread', 'Chicken meat', 'Olive oil', 'Soy sauce', 'Carrot', 'Orange']
  Similar Meal: ['Couscous', 'Rabbit Meat', 'Butter', 'Barbecue sauce', 'Carrot', 'Orange']

Day 3:
  Meal: ['Rice', 'Beef', 'Olive oil', 'Tomato sauce', 'Carrot', 'Orange']
  Similar Meal: ['Millet', 'Roast beef', 'Butter', 'Tomato paste', 'Carrot', 'Orange']

Day 4:
  Meal: ['Pasta', 'Mozzarella', 'Olive oil', 'Mustard', 'Fennel', 'Kiwifruit']
  Similar Meal: ['Couscous', 'Cottage cheese', 'Butter', 'Barbecue sauce', 'Fennel', 'Kiwifruit']

Day 5:
  Meal: ['Pasta', 'Chicken meat', 'Olive oil', 'Barbecue sauce', 'Carrot', 'Orange']
  Similar Meal: ['Couscous', 'Rabbit Meat', 'Butter', 'Tomato sauce', 'Carrot', 'Orange']

Day 6:
  Meal: ['Pasta', 'Ground beef', 'Olive oil', 'Tomato sauce', 'Carrot', 'Orange']
  Similar Mea